Heart Rate from FACE Detection 

In [5]:
import cv2
import numpy as np
from scipy.signal import butter, filtfilt, find_peaks
from scipy.fft import fft
import matplotlib.pyplot as plt

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def extract_green_channel(frame):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        face_roi = frame[y:y+h, x:x+w]
        b, g, r = cv2.split(face_roi)
        return np.mean(g)
    return None

def estimate_heart_rate(signal, fps):
    signal_fft = fft(signal)
    frequencies = np.fft.fftfreq(len(signal_fft), d=1/fps)
    
    freq_range = (frequencies >= 0.7) & (frequencies < 4) 
    filtered_fft = np.abs(signal_fft[freq_range])
    filtered_frequencies = frequencies[freq_range]
    
    if np.sum(filtered_fft) > 0:
        mode_freq = filtered_frequencies[np.argmax(filtered_fft)]
    else:
        mode_freq = 0

    heart_rate_bpm = mode_freq * 60
    return heart_rate_bpm
def main():
    cap = cv2.VideoCapture('Samples\\random.mp4')
    fps = 30
    window_size = 50

    signal_buffer = []
    heart_rates = []
    heart_rates_2 = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        heart_rate = extract_green_channel(frame)

        if heart_rate is not None:
            signal_buffer.append(heart_rate)
            heart_rates.append(heart_rate)
            if len(signal_buffer) > window_size:
                signal_buffer.pop(0)

            if len(signal_buffer) == window_size:
                filtered_signal = bandpass_filter(signal_buffer, 0.7, 4, fps, order=5)
                heart_rate_bpm = estimate_heart_rate(filtered_signal, fps)
                heart_rates_2.append(heart_rate_bpm)

        cv2.namedWindow('Video Feed', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Video Feed', 300, 500)
        cv2.imshow('Video Feed', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    print(heart_rates)
    if heart_rates:
        average_heart_rate = np.mean(heart_rates)
        print(f"Average heart rate: {average_heart_rate}")


    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


[97.4815756249474, 98.07163214455161, 96.18384175206059, 96.92121566972128, 96.78656630304982, 97.66419957559133, 97.0329028903662, 97.09658668821436, 99.69401283757198, 97.7967908942588, 97.57931012093174, 97.87462262817795, 97.3953546167706, 96.46264403292182, 95.19012875233196, 94.92266201945061, 95.15641153435386, 96.10291455341967, 93.76872869874673, 94.53195978884278, 97.72582375530148, 95.5857254655322, 95.30684843017363, 94.727209952443, 95.95086151655393, 94.73190687152417, 94.46761038160685, 95.67783491339725, 97.03924067500111, 96.54776258632212, 95.45809540761837, 96.03464658128995, 95.99784650916014, 96.09208687480277, 95.8961166914141, 96.33669421657027, 97.39666776859504, 96.87282131787626, 96.55784020869378, 96.87925258816598, 96.32268237432564, 95.96247862981112, 96.97381371481674, 96.34502743484225, 97.34340212140307, 97.128874673849, 98.22457546397334, 97.45770323364803, 97.93109904640205, 97.73863085669214, 97.74174227758606, 96.59336908284024, 98.43244919659735, 97